In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime, date
import pickle
from time import time
from src.utils import libSort
import calendar
from datetime import datetime as dpa
from calendar import monthrange
from src.utils.convert_data_type import convertList, convertToHour
from src.utils.QueryData import GetPhone, checkNumberDayRemaining, SearchInfor, GetMinute, GetIndex
from src.utils.mergeToDataFrame import format_time

database = pd.read_csv('/home/ai_car/AthenaBow/data/TotalData.csv')
xgbModel = pickle.load(open('/home/ai_car/AthenaBow/model/24-12/xgb.sav', 'rb'))
# database = database.drop(columns=['STT', 'CallID', 'Hotline', 'QC User Say', 'QC Intent', 'Đánh giá Intent', 'Tỷ lệ nhận diện STT', 'Đánh giá STT', 'Audio', 'Đơn giá', 'CUSTOMER'])
# database = database.drop(columns=['Intent', 'Độ tự tin (%)', 'Entities', 'Entity Value', 'QC Độ trễ', 'Số khách hàng', 'Mã kết thúc bot'])
# database = database.drop(columns=['Action', 'Bot Process (ms)', 'TTS Process (ms)', 'User Say', 'Time in call', 'ASR Process (ms)'])
# database = database.drop(columns=['TENURE', 'RELATIONSHIP_2', 'PHONE_REF3', 'NAME_3', 'RELATIONSHIP_3', 'PERMNENT_PROVINCE', 'RELATIONSHIP_1', 'PHONE_REF2', 'NAME_2'])
# database = database.drop(columns=['MOB', 'DPD', 'REMAINING_PEROID'])
# database = database.drop(columns=['NAME_1', 'DISBURSALDATE', 'PHONE_REF1', 'MAX_DPD', 'DUEDAY', 'FIRST_PAID_DATE'])
# database = database.drop(columns=['Mã tổng đài', 'AMOUNT_FINANCE', 'GENDER', 'Thời lượng gọi'])
# database = database.drop_duplicates(ignore_index= True)
# database = database.dropna(axis=1)
# database = database.reset_index(drop=True)
def convertPhone(database):
    listPhone = []
    for i in tqdm(range(len(database['CURRES_PHONE']))):
        if str(database['CURRES_PHONE'][i]) == "nan":
            listPhone.append(0)
        else:
            listPhone.append(int(database['CURRES_PHONE'][i]))
    return listPhone
database['CURRES_PHONE'] = convertPhone(database)
def encodeTimeCall(data):
    listEncode = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        a = dpa.strptime(data['Thời điểm gọi'][i], '%Y-%m-%d %H:%M:%S')
        listEncode.append(a)
    return listEncode
database['Thời điểm gọi'] = encodeTimeCall(database)
def convertToTime(data):
    hourToCall = []
    for i in tqdm(range(len(data['Thời điểm gọi']))):
        hourToCall.append(convertToHour(data, i))
    return hourToCall
database['TimeCall'] = convertToTime(database)
listTimeEncode = []
for i in tqdm(range(len(database['TimeCall']))):
    listTimeEncode.append(int(int(database['TimeCall'][i]) * 60) + GetMinute(database, i))
database['TimeCall'] = listTimeEncode
database.rename(columns = {'Trạng thái':'Result'}, inplace = True)
def labelEncoder(data):
    encoderLabel = []
    for i in tqdm(range(len(data['Result']))):
        if data['Result'][i] == 'Không thành công' or data['Result'][i] == 'Lỗi':
            encoderLabel.append(0)
        elif data['Result'][i] == 'Nghe máy':
            encoderLabel.append(1)
    return encoderLabel

/home/ai_car/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (8,9,13,14,15,17,18,25,36,37,40,43,48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
100%|██████████| 1699122/1699122 [00:18<00:00, 94293.23it/s] 


In [11]:
listAgreedmentID = 2014401
#2523722
                    # 2662516,
                    # 1661663,
                    # 2599746,
                    # 2421024]

listAgreedmentID = 2599746 #1910188
# for i in range(len(listAgreedmentID)):
listID = GetIndex(data= database, id= listAgreedmentID)
Phone, Product, Nc = SearchInfor(data= database, index= listID[0], NCIndex= listID[-1])
historyCall = GetPhone(data=database, Phone= Phone)
NowDay, LimitInMonth, NowMonth = checkNumberDayRemaining()
whatInsideToModel = [[Phone, Product, Nc]]
listResult = pd.DataFrame()
for i in tqdm(range(len(whatInsideToModel))):
    for j in range(NowDay, LimitInMonth + 1):
        perCent = []
        listTimeAvailabel = []
        for k in tqdm(range(480, 12001, 15)):
            preds = xgbModel.predict_proba([[whatInsideToModel[i][0],
                                            whatInsideToModel[i][1],
                                            whatInsideToModel[i][2],
                                            k,
                                            j,
                                            NowMonth]])
            preds = preds.tolist()
            perCent.append(preds[0][1])
            listTimeAvailabel.append(k)
        whenIsGoodTime = libSort.nlargest(10,zip(perCent, listTimeAvailabel))
        whenIsGoodTime = convertList(whenIsGoodTime)
        StandardScaleTime = []
        for ff in range(len(whenIsGoodTime)):
            HereWeLive = (f"{round(float(whenIsGoodTime[ff][0] * 100),3)}", whenIsGoodTime[ff][1])
            StandardScaleTime.append(HereWeLive)
        DayinWeeks = date(year= datetime.date(datetime.now()).year,
                            month=NowMonth,
                            day= int(j))
        listResult[f'{j}/{NowMonth}/{datetime.date(datetime.now()).year} - {calendar.day_name[DayinWeeks.weekday()]}'] = StandardScaleTime


100%|██████████| 1/1 [00:07<00:00,  7.87s/it]


In [12]:
listResult

,27/12/2022 - Tuesday,28/12/2022 - Wednesday,29/12/2022 - Thursday,30/12/2022 - Friday,31/12/2022 - Saturday
0,"(86.163, 930)","(88.94, 930)","(87.629, 945)","(86.275, 705)","(85.37, 705)"
1,"(86.078, 945)","(88.7, 945)","(86.49, 930)","(85.767, 915)","(85.019, 690)"
2,"(85.194, 885)","(88.496, 885)","(85.963, 795)","(85.668, 885)","(84.626, 900)"
3,"(84.815, 795)","(88.163, 795)","(85.909, 705)","(85.466, 690)","(84.511, 945)"
4,"(84.471, 705)","(88.159, 900)","(85.885, 885)","(85.358, 900)","(84.488, 885)"
5,"(84.229, 915)","(87.83, 840)","(85.58, 900)","(85.036, 795)","(84.327, 795)"
6,"(84.009, 960)","(87.793, 855)","(85.567, 960)","(84.81, 780)","(84.204, 675)"
7,"(83.985, 855)","(87.716, 915)","(85.249, 780)","(84.688, 765)","(83.99, 780)"
8,"(83.949, 690)","(87.579, 705)","(85.211, 915)","(84.688, 750)","(83.083, 915)"
9,"(83.934, 840)","(87.377, 825)","(85.082, 690)","(84.688, 735)","(82.817, 975)"


In [15]:
maxPercent = float(85)
Statictis = []
for m in listResult.columns:
    flag = True
    flag2 = False
    countTotals = 0
    countReach = 0
    topFit = []
    for n in range(len(listResult[m])):
        countTotalsInTime = 0
        countReachInTime = 0
        if float(listResult[m][n][0]) < maxPercent:
            if flag2 == False:
                flag = False
                break
            else:
                flag == True
        else:
            flag = True
            # flag2 = False
            extractDay = int(m.split('/')[0])
            for p in range(len(historyCall)):
                dayExecuse = int(database['Thời điểm gọi'][historyCall[p]].day)
                if dayExecuse == extractDay:
                    countTotals = countTotals + 1
                    if int(listResult[m][n][1]) in range(int(database['TimeCall'][historyCall[p]]) - 15,int(database['TimeCall'][historyCall[p]]) + 15):
                        countTotalsInTime = countTotalsInTime + 1
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReachInTime = countReachInTime + 1
                            countReach = countReach + 1
                            flag2 = True
                    else:
                        if int(database['Result'][historyCall[p]]) == 1:
                            countReach = countReach + 1
                            flag2 = True
        if flag2 == True:
            if countTotalsInTime == 0:
                BestofDay = {'Time to Call' : format_time(listResult[m][n][1]),
                                'Total Call' : countTotals,
                                'Total Reach In Day' : countReach,
                                'Total Call In Time' : int((countTotals/24)*100),
                                'Total Reach In Time' : countReachInTime,
                                'ConfidentInDays' : (countReach/countTotals) * 100,
                                'ConfidentInTimes' : 0}
            else:
                BestofDay = {'Time to Call' : format_time(listResult[m][n][1]),
                                'Total Call' : countTotals,
                                'Total Reach In Day' : countReach,
                                'Total Call In Time' : int((countTotals/24)*100),
                                'Total Reach In Time' : int((int((countTotals/24)*100)*listResult[m][n][0])/100),
                                'ConfidentInDays' : (countReach/countTotals) * 100,
                                'ConfidentInTimes' : listResult[m][n][0]}
            topFit.append(BestofDay)
    if flag == True:
        ResultIHope = {str(m) : topFit}
        Statictis.append(ResultIHope)
    elif flag == False :
        ResultIHope = {str(m) : "Don't found suitable data"}
        Statictis.append(ResultIHope)
        

In [16]:
Statictis

[{'27/12/2022 - Tuesday': "Don't found suitable data"},
 {'28/12/2022 - Wednesday': []},
 {'29/12/2022 - Thursday': []},
 {'30/12/2022 - Friday': "Don't found suitable data"},
 {'31/12/2022 - Saturday': "Don't found suitable data"}]

In [24]:
for i in range(len(historyCall)):
    if database['Thời điểm gọi'][historyCall[i]].day == 31:
        print(1)

1
1
1
1


In [8]:
database['Thời điểm gọi'][historyCall[0]].day

26